In [2]:
from lorem_new import LOREM
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from collections import defaultdict

ModuleNotFoundError: No module named 'SuperLore'

In [ ]:
train_set = pd.read_csv('train_set.csv')
train_label = pd.read_csv('train_label.csv')
test_set = pd.read_csv('test_set.csv')
test_label = pd.read_csv('test_label.csv')
train_set.pop('Unnamed: 0')
train_label.pop('Unnamed: 0')
test_set.pop('Unnamed: 0')
test_label.pop('Unnamed: 0')

: 

In [ ]:
rf = RandomForestClassifier(bootstrap=False, class_weight='balanced',
                       criterion='gini', max_depth=500, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=10,
                       min_samples_split=50, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=None, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)
rf.fit(train_set, train_label)



: 

In [ ]:
predictions1 = rf.predict(train_set)
score = rf.score(train_set, train_label)
report = classification_report(train_label, predictions1)
print('Classification report for train set ', report)
predictions1 = rf.predict(test_set)
score = rf.score(test_set, test_label)
report = classification_report(test_label, predictions1)
print('Classification report for test set ', report)


: 

In [ ]:
neigh_type = 'cfs' # the generation you want (random, genetic, geneticp, cfs, rndgen)
binary = 'binary_from_dts' #how to merge the trees (binary from dts, binary from bb are creating a binary tree, nari is creating a n ari tree)
cxpb = 0.7 # values to set for the genetic generation
mutpb = 0.5 #values to set for the genetic generation
ngen = 2 # number of neighborhood generations to do
runs = 2 #how many neighbours and trees to create and then merge
class_name = 'class' #name of the column used as target
class_values = [0,1] #values that the target may have

feature_names = test_set.columns
real_feature_names = test_set.columns
numeric_columns = list()
for c in test_set.columns:
    numeric_columns.append(test_set.columns.get_loc(c))

features_map = defaultdict(dict)
i = 0
j = 0

while i < len(feature_names) and j < len(real_feature_names):
    if feature_names[i] == real_feature_names[j]:
        features_map[j][feature_names[i].replace('%s=' % real_feature_names[j], '')] = i
        i += 1
        j += 1
    elif feature_names[i].startswith(real_feature_names[j]):
        features_map[j][feature_names[i].replace('%s=' % real_feature_names[j], '')] = i
        i += 1
    else:
        j += 1

: 

In [ ]:
neigh_kwargs = {
        "balance": False,
        "sampling_kind": "gaussian",
        "kind": "gaussian_global",
        "downward_only": True,
        "redo_search": True,
        "forced_balance_ratio": 0.5,
        "cut_radius": True,
        "n": 800,
        "normalize": 'minmax',
        "forced_balance_ratio": 0.5,
        "n_batch": 5,
        "datas": train_set.values
    }

: 

In [ ]:
# neigh type: the way in which you want to generate the neighbourhood (random, genetic, geneticp, rndgen, rndgenp, cfs)
# ocr, mutpb, cxpb: values to set for the genetic generation
# negen: number of neighborhood generations to do
# runs: how many neighbours and trees to create and then merge
# discretize: in the surrogate trees, discretize variables to better generalize and have a smaller tree
# binary: how to merge the trees (binary from dts, binary from bb are creating a binary tree, nari is creating a n ari tree)
# encdec: if you want to generate the neighbours in an encoded space (for now you can select onw hot and target encoding)
# dataset: in case you selected an encdec, this dataset is used to fit the encoder
# K_transformed: a piece of dataset in the form you can feed to the BB
# extreme fidelity: if True, it checks that the prediction of the surrogate model is the same as the BB
# filter c rules: it filters the counterfactual rules by checking if they are true conterfactuals or not
explainer = LOREM(train_set.values, rf.predict, rf.predict_proba, feature_names, class_name, class_values, numeric_columns, features_map,
                      neigh_type=neigh_type, categorical_use_prob=True, continuous_fun_estimation=True, size=1000,
                      ocr=0.1, multi_label=False, one_vs_rest=False, random_state=42, verbose=True,
                      Kc=train_set, bb_predict_proba=rf.predict_proba, K_transformed=train_set, discretize=True,
                      encdec=None, binary=binary, **neigh_kwargs)

: 

In [ ]:
# x the instance to explain
# samples the number of samples to generate during the neighbourhood generation
# use weights True or False
# metric default is neuclidean, it is the metric employed to measure the distance between records
# runs number of times the neighbourhood generation is done
# exemplar_num number of examplars to retrieve
# kwargs a dictionary in which add the parameters needed for cfs generation
# kernel and kernel width are for the definition of weights. deafult is None, it automatically select them.

explanation = explainer.explain_instance_stable(test_set.loc[0].values.reshape(1,-1), 150, runs=runs, n_jobs=2)

: 

In [ ]:
print(explanation.rule)
for c in explanation.crules:
    print(c)

: 

In [ ]:
explanation.exemplars

: 

In [ ]:
explanation.cexemplars

: 

In [ ]:
explanation.feature_importance_all

: 

In [ ]:
explanation.feature_importance

: 

: 